In [ ]:
import tensorflow as tf
import keras
import os
import sys

sys.path.append(os.path.abspath("../"))
from imagenet2012_utils import ImageNetDataset

In [ ]:
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
  except RuntimeError as e:
    print(e)

In [ ]:
(training_images, training_labels) ,\
(validation_images, validation_labels) = \
ImageNetDataset.load_validation_dataset(mode='efficientnetb0')

In [ ]:
inputs = tf.keras.layers.Input(shape=(224,224,3))
model = tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3),
                                        include_top=True,
                                        weights='imagenet')(inputs)
model = tf.keras.Model(inputs=inputs, outputs=model)
model.compile(optimizer='SGD', 
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

model.summary()

In [ ]:
MODEL_PATH = "../models_data/efficientnetB0/"
os.makedirs(MODEL_PATH, exist_ok=True)
model.save(MODEL_PATH+'efficientnetB0_fp32_imagenet2012.keras')

In [ ]:
model.evaluate(validation_images, validation_labels)
model.evaluate(validation_images[:2000], validation_labels[:2000])

In [ ]:
def representative_dataset():
  for images in tf.data.Dataset.from_tensor_slices(validation_images).batch(1).take(1000):
    yield [images]

loaded_model = keras.saving.load_model("../models_data/efficientnetB0/efficientnetB0_fp32_imagenet2012.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()
with open('../models_data/efficientnetB0/efficientnetB0_uint8_imagenet2012.tflite', 'wb') as f:
  f.write(tflite_quant_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()
with open('../models_data/efficientnetB0/efficientnetB0_int8_imagenet2012.tflite', 'wb') as f:
  f.write(tflite_quant_model)